<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/3_5_pipeline_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importamos las librerias

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier


# Metricas
from sklearn.metrics import accuracy_score

# importing pipes for making the Pipe flow
from sklearn.pipeline import Pipeline



In [ ]:
# pasos 1. PC, 2 estandarización, 3. decision tree

pipe = Pipeline([('paso_pca', PCA(n_components = 2)), ('paso_estandarizacion', StandardScaler()),  ('paso_decision_tree', DecisionTreeClassifier())], verbose = True)

# COn  fit aplicamos todo

pipe.fit(X_train, y_train)


print(accuracy_score(y_test, pipe.predict(X_test)))

[Pipeline] .......... (step 1 of 3) Processing paso_pca, total=   0.0s
[Pipeline]  (step 2 of 3) Processing paso_estandarizacion, total=   0.0s
[Pipeline]  (step 3 of 3) Processing paso_decision_tree, total=   0.0s
0.8933333333333333


In [ ]:
pipe = Pipeline([('paso_decision_tree', DecisionTreeClassifier())], verbose = True)

# fitting the data in the pipeline
pipe.fit(X_train, y_train)

print(accuracy_score(y_test, pipe.predict(X_test)))

[Pipeline]  (step 1 of 1) Processing paso_decision_tree, total=   0.0s
0.9466666666666667


In [ ]:
# cargamos el dataset de prueba

iris = datasets.load_iris()
X = iris.data
y = iris.target

#Partimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50)

# paso 1 estandarización, paso 2 clasificadro

pipe = Pipeline([('paso_estandarizacion', StandardScaler()), ('paso_decision_tree', DecisionTreeClassifier())], verbose = True)

# Con fit aplica todos los pasos

pipe.fit(X_train, y_train)

# mostramos los resultados
print(accuracy_score(y_test, pipe.predict(X_test)))

[Pipeline]  (step 1 of 2) Processing paso_estandarizacion, total=   0.0s
[Pipeline]  (step 2 of 2) Processing paso_decision_tree, total=   0.0s
0.96


In [ ]:
### Tutorial Avanzado: Uso de Custom Transformers en Python para Personalizar la Preparación de Datos

# Introducción
# En este tutorial aprenderás a crear y utilizar transformadores personalizados (Custom Transformers) en Python
# utilizando la biblioteca `scikit-learn`. Estos transformadores permiten personalizar la preparación de datos
# en pipelines y adaptarla a necesidades específicas.

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Crear un dataset de ejemplo
def generar_datos():
    np.random.seed(42)
    n = 1000
    X = pd.DataFrame({
        'caracteristica_1': np.random.uniform(0, 100, n),
        'caracteristica_2': np.random.uniform(0, 50, n),
        'caracteristica_3': np.random.normal(50, 10, n),
    })
    # Etiqueta con una relación no lineal para propósitos didácticos

    y = (X['caracteristica_1'] + X['caracteristica_2'] + np.random.normal(0, 5, n) > 100).astype(int)
    return X, y

# Generar el dataset
X, y = generar_datos()


# Dividir los datos
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size=0.3, random_state=42)



# ---- CREACIÓN DE UN CUSTOM TRANSFORMER ----
class GenerarVariablesPersonalizadas(BaseEstimator, TransformerMixin):
    """
    Un transformador personalizado para generar nuevas variables basadas en las columnas existentes.
    """
    def __init__(self, crear_interacciones=True):
        self.crear_interacciones = crear_interacciones

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        # Crear nuevas columnas
        X['suma_caracteristicas'] = X['caracteristica_1'] + X['caracteristica_2']
        X['diferencia_caracteristicas'] = X['caracteristica_1'] - X['caracteristica_3']

        # Crear interacciones si está habilitado
        if self.crear_interacciones:
            X['producto_caracteristicas'] = X['caracteristica_1'] * X['caracteristica_2']
        return X

# ---- CREAR UN PIPELINE COMPLETO ----
pipeline = Pipeline([
    ('generador', GenerarVariablesPersonalizadas(crear_interacciones=True)),
    ('escalador', StandardScaler()),
    ('modelo', LogisticRegression())
])

# Entrenar el pipeline
pipeline.fit(X_entreno, y_entreno)

# Evaluar el modelo
y_pred = pipeline.predict(X_prueba)
precision = accuracy_score(y_prueba, y_pred)

print(f"Precisión del modelo con custom transformer: {precision:.4f}")

# ---- COMPARAR CON UN MODELO SIN TRANSFORMADOR PERSONALIZADO ----
modelo_basico = Pipeline([
    ('escalador', StandardScaler()),
    ('modelo', LogisticRegression())
])

# Entrenar el pipeline básico
modelo_basico.fit(X_entreno, y_entreno)

# Evaluar el modelo básico
y_pred_basico = modelo_basico.predict(X_prueba)
precision_basico = accuracy_score(y_prueba, y_pred_basico)

print(f"Precisión del modelo sin custom transformer: {precision_basico:.4f}")

# ---- RESUMEN ----
print("\nResumen del impacto del Custom Transformer:")
print(f"Precisión con custom transformer: {precision:.4f}")
print(f"Precisión sin custom transformer: {precision_basico:.4f}")

# Conclusión
# Este tutorial muestra que los Custom Transformers permiten agregar flexibilidad y potencia al preprocesamiento
# de datos en pipelines, lo que puede mejorar el rendimiento de los modelos.


Precisión del modelo con custom transformer: 0.9600
Precisión del modelo sin custom transformer: 0.9567

Resumen del impacto del Custom Transformer:
Precisión con custom transformer: 0.9600
Precisión sin custom transformer: 0.9567
